# CCA Analysis 2: conducting the experiments

In this file, CCA analysis is run with all different types of datasets, namely those in all_datasets_2.p (made using make_all_datasets() in clustering.ipynb, where if use_all_els = False, then make_data() used elec_nrs_outcome if the time window type was AFTER_OUTCOME and elec_nrs_gamebtn if the time window type was GAMEPRES_BUTTONPRESS).

In [1]:
from data_reader_writer import * 
import data_process as dp
import itertools as it
import pandas as pd
import numpy as np
from sklearn.cross_decomposition import CCA
from collections import defaultdict
from scipy.stats import pearsonr
from sklearn.model_selection import cross_val_score
import operator
from difflib import SequenceMatcher
import pickle

Load in the data and copy-edit necessary functions from cca_analysis.

In [2]:
ephys_data = load_ephys_data()
gamble_choices = load_gamble_choices_data()
behavior_data = load_behavior_data()
regressor_data = load_regressor_data()

In [3]:
all_datasets = load_var("all_datasets_2")

## Experiment functions

In [6]:
# Time-locks the given ephys trials to [-i1;i2]ms from the outcome reveal
# NB! The ephys data has to be buttonpress_hg!
def time_lock_outcome(ephys_data, behav_data, i1, i2):
    result = []
    experiment = [item for sublist in ephys_data for item in
                  sublist]  # the measurements of the entire experiment in one list
    # should have n_trials x 3001 elements

    for index, row in behav_data.iterrows():
        delay = round((row['reveal.time'] - row['buttonpress.time']) 
                      * 1000)  # the time between buttonpress event and outcome reveal in ms
        outcome_moment = index * 3001 + 1000 + delay  # the buttonpress is 1s = 1000ms from the beginning of the trial
        if outcome_moment + i2 >= len(experiment):
            trial = experiment[outcome_moment - i1:]
        else:
            trial = experiment[outcome_moment - i1:outcome_moment + i2]
        result.append(trial)

    return result

In [7]:
# Performs cross-validation with CCA, saving the train and test set correlations, weighted sums (both X and Y) and the weights
# of X and Y for all folds
def cross_validation(X, Y, k=5):
    X_folds = np.array(np.array_split(X, k))
    Y_folds = np.array(np.array_split(Y, k))

    results = {}
    test_corrs = []
    train_corrs = []
    x_weights = []
    y_weights = []
    best_test_corr = -1
    best_train_corr = -1
    best_X_testsums = None
    best_Y_testsums = None
    best_X_trainsums = None
    best_Y_trainsums = None
    
    for i in range(k):
        cca = CCA(n_components=1)
        
        X_train = np.array([lst for j in range(k) if i != j for lst in X_folds[j]])
        Y_train = np.array([lst for j in range(k) if i != j for lst in Y_folds[j]])
        
        X_test = X_folds[i]
        Y_test = Y_folds[i]
        
        cca.fit_transform(X_train, Y_train)
        a = cca.x_weights_.flatten()
        b = cca.y_weights_.flatten()
        
        aX_test = a.dot(X_test.T)
        bY_test = b.dot(Y_test.T)
        aX_train = a.dot(X_train.T)
        bY_train = b.dot(Y_train.T)
        
        test_corr = np.corrcoef(aX_test.T, bY_test.T)[0][1]
        train_corr = np.corrcoef(aX_train.T, bY_train.T)[0][1]
        
        test_corrs.append(test_corr)
        train_corrs.append(train_corr)
        x_weights.append(a)
        y_weights.append(b)
        
        if test_corr > best_test_corr:
            best_test_corr = test_corr
            best_X_testsums = aX_test
            best_Y_testsums = bY_test
        
        if train_corr > best_train_corr:
            best_train_corr = train_corr
            best_X_trainsums = aX_train
            best_Y_trainsums = bY_train
        
    
    results["test_corrs"] = test_corrs
    results["train_corrs"] = train_corrs
    results["x_weights"] = x_weights
    results["y_weights"] = y_weights
    results["sums_X_test"] = best_X_testsums
    results["sums_Y_test"] = best_Y_testsums
    results["sums_X_train"] = best_X_trainsums
    results["sums_Y_train"] = best_Y_trainsums
    return results

In [109]:
# X_data - the ephys dataset from all_datasets_2.p, has to be a dictionary in the form 
# (subject, tw_type, trial_type, tw_per_el, boolean) -> dataset
# trial_type (str) - the type of trials included in the experiment
# tw_type (str) - time window type
# regressors (list<str>) - the Y features used in CCA
# exp_nr (int) - the number of the experiment
# tw_per_el (int) - time windows per electrode 
# use_all_els - if True, then use all electrodes of the patient, otherwise use the best ones
# elec_nrs (map<str, list<int>>) - mapping of subject -> list of electrodes which to use in X
# verbose - if True, then print stuff, otherwise don't

# Runs cross-validation for every subject with the given set of parameters
def experiment2(X_data, trial_type, tw_type, regressors, exp_nr, tw_per_el=1, use_all_els=True, k=5, elec_nrs=None,
               verbose=False):
    results = defaultdict(dict)
    n_regr = len(regressors)
    results["Y_features"] = regressors
    results["nr"] = exp_nr
    has_prev_trial_features = any(map(lambda s: s.startswith("previous_"), regressors))
    
    for s in SUBJECTS:
        if verbose:
            print(s)
            
        X = X_data[(s, tw_type, trial_type, tw_per_el, use_all_els)]
        if elec_nrs != None:
            start_idx = [(nr - 1) * tw_per_el for nr in elec_nrs[s]] # nr - 1 bc elec_nrs are from 1 to n
            elec_idx = np.array([[i + j for j in range(tw_per_el)] for i in start_idx]).flatten()
            X = X[:, elec_idx]  # select only necessary electrodes
        
        n_elec = N_ELEC[s]
        # Filter out trials that were not included in the ephys data from behav data
        s_behav = behavior_data[s][behavior_data[s]['trial.included'] == 1]
        s_behav_reidx = s_behav.reset_index(drop=True) # so that the indices would be 0, ... , len(s_ephys) - 1
        
        # Get the correct trial indices for the trial type
        indices = None  # The indices from the reindexed behav data
        indices_orig = None  # The indices from the original behav data
        if trial_type == LOSS:
            indices = s_behav_reidx[s_behav_reidx['outcome'] == 'Loss'].index
            indices_orig = s_behav[s_behav['outcome'] == 'Loss'].index
        elif trial_type == GAMBLE:
            indices = s_behav_reidx[s_behav_reidx['choice.class'] == 'Gamble'].index
            indices_orig = s_behav[s_behav['choice.class'] == 'Gamble'].index
        elif trial_type == ALL:
            indices = s_behav_reidx.index
            indices_orig = s_behav.index
            
        # If we have any previous trial features, we can't include the first trial, which doesn't have a previous trial    
        # We check if 0 is in indices_orig because indices always begins with 0
        if has_prev_trial_features and 0 in indices_orig:
            assert indices[0] == 0 and indices_orig[0] == 0, "Wrong index ordering"
            indices = indices[1:]
            indices_orig = indices_orig[1:]
            
        n_trials = len(indices)
        # Matrix in the shape (n_regressors x n_trials)
        Y = np.zeros([n_trials, n_regr])
        
        # For every regressor, fill a column with its correct values
        for i in range(n_regr):
            if regressors[i] == "prize":
                # We must use indices_orig because the length of gamble_choices is 200 -- it contains all trial information
                Y[:, i] = gamble_choices.iloc[indices_orig]['Risky.Bet.Amount']
            elif regressors[i] == "previous_prize":
                # Use indices_org - 1 bc we want the previous trials' indices
                Y[:, i] = gamble_choices.iloc[indices_orig - 1]['Risky.Bet.Amount']
            else:
                # We must use indices because the length of regressor data is the length of ephys data -- only included trials
                Y[:, i] = regressor_data[s][regressors[i]].flatten()[indices]
                
        if has_prev_trial_features and X.shape[0] == Y.shape[0] + 1:
            X = X[1:]
    
        cca = CCA(n_components=1)
        cca.fit_transform(X, Y)
        a = cca.x_weights_.flatten()
        b = cca.y_weights_.flatten()

        aX = a.dot(X.T)    
        bY = b.dot(Y.T)
        if verbose:
            print("a:", a)
            print("b:", b)
            print("Corr for the whole dataset:", np.corrcoef(aX.T, bY.T)[0][1])
            
        results[s]["a"] = a
        results[s]["b"] = b
        results[s]["coef"] = np.corrcoef(aX.T, bY.T)[0][1]
        
        results[s]["cross_val"] = cross_validation(X, Y, k)
        if verbose:
            print("Train corrs:", results[s]["cross_val"]["train_corrs"])
            print("Test corrs:", results[s]["cross_val"]["test_corrs"])
    
    return results

In [84]:
# trial_type (str) - the type of trials included in the experiment
# tw_type (str) - time window type
# regressors (list<str>) - the Y features used in CCA
# exp_nr (int) - the number of the experiment

# Runs the same experiment, but using one electrode at a time, saving the results for all electrodes separately
def experiment_electrodes(X_data, trial_type, tw_type, regressors, exp_nr, use_all_els=True, k=5, verbose=False):
    results = defaultdict(lambda: defaultdict(dict))
    n_regr = len(regressors)
    results["Y_features"] = regressors
    results["nr"] = exp_nr
    has_prev_trial_features = any(map(lambda s: s.startswith("previous_"), regressors))
    
    for s in SUBJECTS:
        if verbose:
            print(s)
            
        X = X_data[(s, tw_type, trial_type, 1, use_all_els)]
            
        n_elec = N_ELEC[s]
        # Filter out trials that were not included in the ephys data from behav data
        s_behav = behavior_data[s][behavior_data[s]['trial.included'] == 1]
        s_behav_reidx = s_behav.reset_index(drop=True) # so that the indices would be 0, ... , len(s_ephys) - 1
        
        # Pick the correct ephys data
        s_ephys = None
        if tw_type == AFTER_OUTCOME:
            s_ephys = ephys_data[s]['buttonpress_events_hg']
        elif tw_type == GAMEPRES_BUTTONPRESS:
            s_ephys = ephys_data[s]['game_events_hg']
        
        # Get the correct trial indices for the trial type
        indices = None  # The indices from the reindexed behav data
        indices_orig = None  # The indices from the original behav data
        if trial_type == LOSS:
            indices = s_behav_reidx[s_behav_reidx['outcome'] == 'Loss'].index
            indices_orig = s_behav[s_behav['outcome'] == 'Loss'].index
        elif trial_type == GAMBLE:
            indices = s_behav_reidx[s_behav_reidx['choice.class'] == 'Gamble'].index
            indices_orig = s_behav[s_behav['choice.class'] == 'Gamble'].index
        elif trial_type == ALL:
            indices = s_behav_reidx.index
            indices_orig = s_behav.index
            
        # If we have any previous trial features, we can't include the first trial, which doesn't have a previous trial    
        # We check if 0 is in indices_orig because indices always begins with 0
        if has_prev_trial_features and 0 in indices_orig:
            assert indices[0] == 0 and indices_orig[0] == 0, "Wrong index ordering"
            indices = indices[1:]
            indices_orig = indices_orig[1:]

        n_trials = len(indices)
        # Matrix in the shape (n_regressors x n_trials)
        Y = np.zeros([n_trials, n_regr])
        
        # For every regressor, fill a column with its correct values
        for i in range(n_regr):
            if regressors[i] == "prize":
                # We must use indices_orig because the length of gamble_choices is 200 -- it contains all trial information
                Y[:, i] = gamble_choices.iloc[indices_orig]['Risky.Bet.Amount']
            elif regressors[i] == "previous_prize":
                # Use indices_org - 1 bc we want the previous trials' indices
                Y[:, i] = gamble_choices.iloc[indices_orig - 1]['Risky.Bet.Amount']
            else:
                # We must use indices because the length of regressor data is the length of ephys data -- only included trials
                Y[:, i] = regressor_data[s][regressors[i]].flatten()[indices]
            

        electrodes = list(range(1, n_elec + 1))
        for el in electrodes:
            X_el = X[:, el - 1].reshape(-1, 1)
            
            if has_prev_trial_features and len(X_el) == len(Y) + 1:
                X_el = X_el[1:]
            
            cca = CCA(n_components=1)
            cca.fit_transform(X_el, Y)
            a = cca.x_weights_.flatten()
            b = cca.y_weights_.flatten()
            
            aX = a.dot(X_el.T)    
            bY = b.dot(Y.T)

            if verbose:
                print("Corr for the whole dataset:", np.corrcoef(aX.T, bY.T)[0][1]) 

            results[s][el]["a"] = a
            results[s][el]["b"] = b
            results[s][el]["coef"] = np.corrcoef(aX.T, bY.T)[0][1]
            results[s][el]["cross_val"] = cross_validation(X_el, Y, k)
            
    return results

In [10]:
# Transform a 'defaultdict' type dictionary to a regular dictionary
def default_to_regular(d):
    if isinstance(d, defaultdict):
        d = {k: default_to_regular(v) for k, v in d.items()}
    return d

## Best electrodes vol 1

In [5]:
# The necessary data was generated in cca_analysis ("One electrode, one regressor"), load it in
# Contains a list of four dictionaries
all_results_el_reg = load_var("all_results_el_reg_1")

In [11]:
# Generates a map where for every subject and every regressor there is the "best" electrode: the electrode which had the highest
# correlation with the regressor on the whole dataset where the time window is 1.5s after outcome reveal
def get_best_elec_outcome_1(results_el_current_reg, results_el_past_reg):
    best_elec_outcome = defaultdict(lambda: defaultdict(int))
    
    for s in SUBJECTS:
        for regr in results_el_current_reg[s]:
            corr_list = sorted(results_el_current_reg[s][regr], reverse=True)
            best_elec_outcome[s][regr] = corr_list[0][1]
    
        for regr in results_el_past_reg[s]:
            corr_list = sorted(results_el_past_reg[s][regr], reverse=True)
            best_elec_outcome[s][regr] = corr_list[0][1]
    
    return best_elec_outcome

In [12]:
# Generates a map where for every subject and every regressor there is the "best" electrode: the electrode which had the highest
# correlation with the regressor on the whole dataset where the time window is between game presentation and button press
def get_best_elec_gamebtn_1(results_el_current_reg, results_el_past_reg):
    best_elec_gamebtn = defaultdict(lambda: defaultdict(int))

    for s in SUBJECTS:
        for regr in results_el_current_reg[s]:
            corr_list = sorted(results_el_current_reg[s][regr], reverse=True)
            best_elec_gamebtn[s][regr] = corr_list[0][1]
    
    for regr in results_el_past_reg[s]:
        corr_list = sorted(results_el_past_reg[s][regr], reverse=True)
        best_elec_gamebtn[s][regr] = corr_list[0][1]
        
    return best_elec_gamebtn

In [16]:
# Gets the best electrodes VERSION 1: for every subject: get the one best electrode for every single regressor
# Use the best_elec_outcome_1 or best_elec_gamebtn_1 depending on the time window type
# Therefore, if k regressors then max k electrodes (can be less than k because some electrode might be the best for several
# regressors)
def get_best_electrodes_1(regressors, tw_type):
    elec_nrs = defaultdict(set)
    
    if tw_type == AFTER_OUTCOME:
        for s in SUBJECTS:
            for regr in regressors:
                elec_nrs[s].add(best_elec_outcome_1[s][regr])
    
    elif tw_type == GAMEPRES_BUTTONPRESS:
        for s in SUBJECTS:
            for regr in regressors:
                elec_nrs[s].add(best_elec_gamebtn_1[s][regr])
    
    return elec_nrs

In [15]:
best_elec_outcome_1 = get_best_elec_outcome_1(all_results_el_reg[0], all_results_el_reg[2])
best_elec_gamebtn_1 = get_best_elec_gamebtn_1(all_results_el_reg[1], all_results_el_reg[3])

In [97]:
best_elec_outcome_1['s1'].keys()

dict_keys(['exputil', 'gamble_ind', 'loss_ind', 'regret', 'risk', 'rpe', 'win_ind', 'winprob', 'previous_exputil', 'previous_gamble_ind', 'previous_loss_ind', 'previous_regret', 'previous_risk', 'previous_rpe', 'previous_win_ind', 'previous_winprob'])

## Best electrodes vol 2

In [100]:
# Generates a map where for every subject and every set of regressors there is a list of electrodes that are sorted by their
# correlation with the set of regressors over the whole dataset (time window being AFTER_OUTCOME)
def get_best_elec_outcome_2(result_list):
    best_elec_outcome_2 = defaultdict(lambda: defaultdict(list))

    for res in result_list: 
        y_features = tuple(res["Y_features"])  # using this as dict key
        for s in SUBJECTS:
            coefs = []
            for el in res[s].keys():
                corr = res[s][el]["coef"]
                coefs.append((corr, el))
            coefs_sorted = sorted(coefs, reverse=True)  # sort by corrs 
            elecs_sorted = list(map(lambda coef_el: coef_el[1], coefs_sorted))  # take electrodes from the tuples
            best_elec_outcome_2[s][y_features] = elecs_sorted
    
    return default_to_regular(best_elec_outcome_2)

In [101]:
# Generates a map where for every subject and every set of regressors there is a list of electrodes that are sorted by their
# correlation with the set of regressors over the whole dataset (time window being GAMEPRES_BUTTONPRESS)
def get_best_elec_gamebtn_2(result_list): 
    best_elec_gamebtn_2 = defaultdict(lambda: defaultdict(list))

    for res in result_list: 
        y_features = tuple(res["Y_features"])  # using this as dict key
        for s in SUBJECTS:
            coefs = []
            for el in res[s].keys():
                corr = res[s][el]["coef"]
                coefs.append((corr, el))
            coefs_sorted = sorted(coefs, reverse=True)  # sort by corrs 
            elecs_sorted = list(map(lambda coef_el: coef_el[1], coefs_sorted))  # take electrodes from the tuples
            best_elec_gamebtn_2[s][y_features] = elecs_sorted
            
    return default_to_regular(best_elec_gamebtn_2)

In [50]:
# n_elec -- how many best electrodes are selected 
# n_elec must be < 6
# p_elec -- percent of electrodes; is used if not None

# Gets the best OUTCOME electrodes VERSION 2: for every subject: get the one best electrode for every single regressor
# Therefore, if k regressors then max k electrodes (can be less than k because some electrode might be the best for several
# regressors)
def get_best_electrodes_2(regressors, tw_type, n_elec=1, p_elec=None):
    elec_nrs = defaultdict(list)
    
    if tw_type == AFTER_OUTCOME:
        for s in SUBJECTS:
            for key in best_elec_outcome_2[s].keys():
                if set(regressors) == set(key):
                    if p_elec is not None:
                        n_elec = int(round(N_ELEC[s] * p_elec))  # the int is necessary for numpy floats
                    
                    elec_nrs[s] = best_elec_outcome_2[s][key][:n_elec]
    
    elif tw_type == GAMEPRES_BUTTONPRESS:
        for s in SUBJECTS:
            for key in best_elec_gamebtn_2[s].keys():
                if set(regressors) == set(key):
                    if p_elec is not None:
                        n_elec = int(round(N_ELEC[s] * p_elec))
                        
                    elec_nrs[s] = best_elec_gamebtn_2[s][key][:n_elec]
    
    return elec_nrs

### Sets of regressors

To use get_best_electrodes_2, we need to have results for every combination of regressors. We will try with every combination of current trial outcome regressors in the AFTER_OUTCOME time window, every combination of previous trial outcome regressors in the AFTER_OUTCOME time window, every combination of current trial choice regressors in the GAMEPRES_BUTTONPRESS time window, and every combination of previous trial choice regressors in the GAMEPRES_BUTTONPRESS time window.

In [18]:
# Finds all subsets of length n in set s
def findsubsets(s, n): 
    return [list(i) for i in it.combinations(s, n)] 

In [19]:
OUTCOME_REGRESSORS = ["loss_ind", "regret", "rpe", "win_ind"]
GAMEBTN_REGRESSORS = ["exputil", "gamble_ind", "risk", "winprob"]

In [74]:
PREV_OUTCOME_REGRESSORS = ["previous_loss_ind", "previous_regret", "previous_rpe", "previous_win_ind"]
PREV_GAMEBTN_REGRESSORS = ["previous_exputil", "previous_gamble_ind", "previous_risk", "previous_winprob"]

In [20]:
findsubsets(OUTCOME_REGRESSORS, 3)

[['loss_ind', 'regret', 'rpe'],
 ['loss_ind', 'regret', 'win_ind'],
 ['loss_ind', 'rpe', 'win_ind'],
 ['regret', 'rpe', 'win_ind']]

In [21]:
outcome_subsets = [OUTCOME_REGRESSORS] + findsubsets(OUTCOME_REGRESSORS, 2) + findsubsets(OUTCOME_REGRESSORS, 3)

In [22]:
gamebtn_subsets = [GAMEBTN_REGRESSORS] + findsubsets(GAMEBTN_REGRESSORS, 2) + findsubsets(GAMEBTN_REGRESSORS, 3)

In [75]:
outcome_prev_subsets = [PREV_OUTCOME_REGRESSORS] + findsubsets(PREV_OUTCOME_REGRESSORS, 2) + findsubsets(
    PREV_OUTCOME_REGRESSORS, 3)

In [76]:
gamebtn_prev_subsets = [PREV_GAMEBTN_REGRESSORS] + findsubsets(PREV_GAMEBTN_REGRESSORS, 2) + findsubsets(
    PREV_GAMEBTN_REGRESSORS, 3)

In [77]:
print(len(all_datasets))
print(len(outcome_subsets))
print(len(gamebtn_subsets))
print(len(outcome_prev_subsets))
print(len(gamebtn_prev_subsets))

480
11
11
11
11


### 1. Run experiment_electrodes() with all combos of current trial regressors

In [35]:
all_experiment_electrodes_ao = []
all_experiment_electrodes_gb = []

exp_nr = 1
for subset in outcome_subsets:
    print(subset)
    res = experiment_electrodes(X_data=all_datasets, trial_type=ALL, tw_type=AFTER_OUTCOME, regressors=subset, exp_nr=exp_nr, 
                      verbose=False)
    exp_nr += 1
    all_experiment_electrodes_ao.append(res)

exp_nr = 1
for subset in gamebtn_subsets:
    print(subset)
    res = experiment_electrodes(X_data=all_datasets, trial_type=ALL, tw_type=GAMEPRES_BUTTONPRESS, regressors=subset, 
                                exp_nr=exp_nr, verbose=False)
    exp_nr += 1
    all_experiment_electrodes_gb.append(res)

['loss_ind', 'regret', 'rpe', 'win_ind']
['loss_ind', 'regret']
['loss_ind', 'rpe']
['loss_ind', 'win_ind']
['regret', 'rpe']
['regret', 'win_ind']
['rpe', 'win_ind']
['loss_ind', 'regret', 'rpe']
['loss_ind', 'regret', 'win_ind']
['loss_ind', 'rpe', 'win_ind']
['regret', 'rpe', 'win_ind']
['exputil', 'gamble_ind', 'risk', 'winprob']
['exputil', 'gamble_ind']
['exputil', 'risk']
['exputil', 'winprob']
['gamble_ind', 'risk']
['gamble_ind', 'winprob']
['risk', 'winprob']
['exputil', 'gamble_ind', 'risk']
['exputil', 'gamble_ind', 'winprob']
['exputil', 'risk', 'winprob']
['gamble_ind', 'risk', 'winprob']


In [39]:
# save_var(list(map(lambda d: default_to_regular(d), all_experiment_electrodes_ao)), "all_experiment_electrodes_ao")
# save_var(list(map(lambda d: default_to_regular(d), all_experiment_electrodes_gb)), "all_experiment_electrodes_gb")

In [27]:
all_experiment_electrodes_ao = load_var("all_experiment_electrodes_ao")
all_experiment_electrodes_gb = load_var("all_experiment_electrodes_gb")

### 2. Run experiment_electrodes() with all combos of previous trial regressors

In [85]:
all_experiment_electrodes_ao_prev = []
all_experiment_electrodes_gb_prev = []

exp_nr = 1
for subset in outcome_prev_subsets:
    print(subset)
    res = experiment_electrodes(X_data=all_datasets, trial_type=ALL, tw_type=AFTER_OUTCOME, regressors=subset, exp_nr=exp_nr, 
                      verbose=False)
    exp_nr += 1
    all_experiment_electrodes_ao_prev.append(res)

exp_nr = 1
for subset in gamebtn_prev_subsets:
    print(subset)
    res = experiment_electrodes(X_data=all_datasets, trial_type=ALL, tw_type=GAMEPRES_BUTTONPRESS, regressors=subset, 
                                exp_nr=exp_nr, verbose=False)
    exp_nr += 1
    all_experiment_electrodes_gb_prev.append(res)

['previous_loss_ind', 'previous_regret', 'previous_rpe', 'previous_win_ind']
['previous_loss_ind', 'previous_regret']
['previous_loss_ind', 'previous_rpe']
['previous_loss_ind', 'previous_win_ind']
['previous_regret', 'previous_rpe']
['previous_regret', 'previous_win_ind']
['previous_rpe', 'previous_win_ind']
['previous_loss_ind', 'previous_regret', 'previous_rpe']
['previous_loss_ind', 'previous_regret', 'previous_win_ind']
['previous_loss_ind', 'previous_rpe', 'previous_win_ind']
['previous_regret', 'previous_rpe', 'previous_win_ind']
['previous_exputil', 'previous_gamble_ind', 'previous_risk', 'previous_winprob']
['previous_exputil', 'previous_gamble_ind']
['previous_exputil', 'previous_risk']
['previous_exputil', 'previous_winprob']
['previous_gamble_ind', 'previous_risk']
['previous_gamble_ind', 'previous_winprob']
['previous_risk', 'previous_winprob']
['previous_exputil', 'previous_gamble_ind', 'previous_risk']
['previous_exputil', 'previous_gamble_ind', 'previous_winprob']
['pre

In [86]:
# save_var(list(map(lambda d: default_to_regular(d), all_experiment_electrodes_ao_prev)), "all_experiment_electrodes_ao_prev")
# save_var(list(map(lambda d: default_to_regular(d), all_experiment_electrodes_gb_prev)), "all_experiment_electrodes_gb_prev")

In [87]:
all_experiment_electrodes_ao_prev = load_var("all_experiment_electrodes_ao_prev")
all_experiment_electrodes_gb_prev = load_var("all_experiment_electrodes_gb_prev")

### 3. Generate maps

In [28]:
best_elec_outcome_2 = get_best_elec_outcome_2(all_experiment_electrodes_ao)
best_elec_gamebtn_2 = get_best_elec_gamebtn_2(all_experiment_electrodes_gb)

In [102]:
best_elec_outcome_prev_2 = get_best_elec_outcome_2(all_experiment_electrodes_ao_prev)
best_elec_gamebtn_prev_2 = get_best_elec_gamebtn_2(all_experiment_electrodes_gb_prev)

In [107]:
# Need to combine these because get_best_electrodes_2 uses best_elec_outcome_2
best_elec_outcome_2 = {**best_elec_outcome_2, **best_elec_outcome_prev_2}
best_elec_gamebtn_2 = {**best_elec_gamebtn_2, **best_elec_gamebtn_prev_2}

## Experiment runs

### 1. All combinations with get_best_electrodes_1

In [113]:
# Runs experiment2 with all datasets using ALL trials, using get_best_electrodes_1 for elec_nrs
def all_combinations_best_elecs_1(datasets, outcome_subsets=outcome_subsets, gamebtn_subsets=gamebtn_subsets):
    results_ao = defaultdict(lambda: defaultdict(dict))
    results_gb = defaultdict(lambda: defaultdict(dict))
    trial_type = ALL
    use_all_els = True
    exp_nr = 1
    
    # 1.
    tw_type = AFTER_OUTCOME
    print(tw_type)
    for subset in outcome_subsets:
        print(subset)
        for tw_per_el in [1, 2, 3]:
            results_ao[tuple(subset)][(tw_per_el, "all")] = experiment2(X_data=datasets, trial_type=trial_type, tw_type=tw_type, 
                                                        regressors=subset, exp_nr=exp_nr, tw_per_el=tw_per_el, 
                                                        use_all_els=use_all_els, k=5, elec_nrs=None, verbose=False)
            
            results_ao[tuple(subset)][(tw_per_el, "best")] = experiment2(X_data=datasets, trial_type=trial_type, tw_type=tw_type, 
                                                        regressors=subset, exp_nr=exp_nr, tw_per_el=tw_per_el, 
                                                        use_all_els=use_all_els, k=5, elec_nrs=get_best_electrodes_1(
                                                            subset, tw_type), verbose=False)
            exp_nr += 1

    exp_nr = 1
    # 2.
    tw_type = GAMEPRES_BUTTONPRESS
    print(tw_type)
    for subset in gamebtn_subsets:
        print(subset)
        for tw_per_el in [1, 2, 3]:
            results_gb[tuple(subset)][(tw_per_el, "all")] = experiment2(X_data=datasets, trial_type=trial_type, tw_type=tw_type, 
                                                        regressors=subset, exp_nr=exp_nr, tw_per_el=tw_per_el, 
                                                        use_all_els=use_all_els, k=5, elec_nrs=None, verbose=False)
            
            results_gb[tuple(subset)][(tw_per_el, "best")] = experiment2(X_data=datasets, trial_type=trial_type, tw_type=tw_type, 
                                                        regressors=subset, exp_nr=exp_nr, tw_per_el=tw_per_el, 
                                                        use_all_els=use_all_els, k=5, elec_nrs=get_best_electrodes_1(
                                                            subset, tw_type), verbose=False)
            exp_nr += 1

    return results_ao, results_gb

**Current trial regressors:**

In [27]:
# results_ao, results_gb = all_combinations_best_elecs_1(all_datasets)

1.5s after outcome reveal
['loss_ind', 'regret', 'rpe', 'win_ind']
['loss_ind', 'regret']
['loss_ind', 'rpe']
['loss_ind', 'win_ind']
['regret', 'rpe']
['regret', 'win_ind']
['rpe', 'win_ind']
['loss_ind', 'regret', 'rpe']
['loss_ind', 'regret', 'win_ind']
['loss_ind', 'rpe', 'win_ind']


C:\Users\MariLiis\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)


['regret', 'rpe', 'win_ind']
between game presentation and buttonpress
['exputil', 'gamble_ind', 'risk', 'winprob']
['exputil', 'gamble_ind']
['exputil', 'risk']
['exputil', 'winprob']
['gamble_ind', 'risk']
['gamble_ind', 'winprob']
['risk', 'winprob']
['exputil', 'gamble_ind', 'risk']
['exputil', 'gamble_ind', 'winprob']
['exputil', 'risk', 'winprob']


C:\Users\MariLiis\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)


['gamble_ind', 'risk', 'winprob']


In [37]:
# save_var((default_to_regular(results_ao), default_to_regular(results_gb)), "all_combinations_best_elecs_1")

In [26]:
results_ao, results_gb = load_var("all_combinations_best_elecs_1")

In [72]:
results_ao[('loss_ind', 'rpe', 'win_ind')].keys()

dict_keys([(1, 'all'), (1, 'best'), (2, 'all'), (2, 'best'), (3, 'all'), (3, 'best')])

In [73]:
results_ao[('loss_ind', 'rpe', 'win_ind')][(1, 'all')].keys()

dict_keys(['Y_features', 'nr', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10'])

**Previous trial regressors:**

In [114]:
# results_ao_prev, results_gb_prev = all_combinations_best_elecs_1(all_datasets, outcome_prev_subsets, gamebtn_prev_subsets)

1.5s after outcome reveal
['previous_loss_ind', 'previous_regret', 'previous_rpe', 'previous_win_ind']
['previous_loss_ind', 'previous_regret']
['previous_loss_ind', 'previous_rpe']
['previous_loss_ind', 'previous_win_ind']
['previous_regret', 'previous_rpe']
['previous_regret', 'previous_win_ind']
['previous_rpe', 'previous_win_ind']
['previous_loss_ind', 'previous_regret', 'previous_rpe']
['previous_loss_ind', 'previous_regret', 'previous_win_ind']
['previous_loss_ind', 'previous_rpe', 'previous_win_ind']
['previous_regret', 'previous_rpe', 'previous_win_ind']
between game presentation and buttonpress
['previous_exputil', 'previous_gamble_ind', 'previous_risk', 'previous_winprob']
['previous_exputil', 'previous_gamble_ind']
['previous_exputil', 'previous_risk']
['previous_exputil', 'previous_winprob']
['previous_gamble_ind', 'previous_risk']
['previous_gamble_ind', 'previous_winprob']
['previous_risk', 'previous_winprob']
['previous_exputil', 'previous_gamble_ind', 'previous_risk']
[

In [115]:
# save_var((default_to_regular(results_ao_prev), default_to_regular(results_gb_prev)), "all_combinations_best_elecs_prev_1")

### 2. All combinations with get_best_electrodes_2

In [89]:
# Uses get_best_electrodes_2 with different percentage of electrodes
def all_combinations_best_elecs_2(datasets, outcome_subsets=outcome_subsets, gamebtn_subsets=gamebtn_subsets):
    results_ao = defaultdict(lambda: defaultdict(dict))
    results_gb = defaultdict(lambda: defaultdict(dict))
    trial_type = ALL
    use_all_els = True
    exp_nr = 1
    
    # 1.
    tw_type = AFTER_OUTCOME
    print(tw_type)
    for subset in outcome_subsets:
        print(subset)
        for tw_per_el in [1, 2, 3]:
            for percent in np.arange(0.2, 1.1, 0.2):
                percent = round(percent, 1) # just in case
                results_ao[tuple(subset)][(tw_per_el, "all", percent)] = experiment2(X_data=datasets, trial_type=trial_type, 
                                                                                     tw_type=tw_type, regressors=subset, 
                                                                                     exp_nr=exp_nr, tw_per_el=tw_per_el, 
                                                                                     use_all_els=use_all_els, k=5, 
                                                                                     elec_nrs=None, verbose=False)

                results_ao[tuple(subset)][(tw_per_el, "best", percent)] = experiment2(X_data=datasets, trial_type=trial_type, 
                                                                                      tw_type=tw_type, regressors=subset, 
                                                                                      exp_nr=exp_nr, tw_per_el=tw_per_el, 
                                                                                      use_all_els=use_all_els, k=5, 
                                                                                      elec_nrs=get_best_electrodes_2(
                                                                                          regressors=subset, tw_type=tw_type, 
                                                                                          p_elec=percent), 
                                                                                      verbose=False)
                exp_nr += 1
                

    exp_nr = 1
    # 2.
    tw_type = GAMEPRES_BUTTONPRESS
    print(tw_type)
    for subset in gamebtn_subsets:
        print(subset)
        for tw_per_el in [1, 2, 3]:
            for percent in np.arange(0.2, 1.1, 0.2):
                percent = round(percent, 1)
                results_gb[tuple(subset)][(tw_per_el, "all", percent)] = experiment2(X_data=datasets, trial_type=trial_type, 
                                                                                     tw_type=tw_type, regressors=subset, 
                                                                                     exp_nr=exp_nr, tw_per_el=tw_per_el, 
                                                                                     use_all_els=use_all_els, k=5, 
                                                                                     elec_nrs=None, verbose=False)

                results_gb[tuple(subset)][(tw_per_el, "best", percent)] = experiment2(X_data=datasets, trial_type=trial_type, 
                                                                                      tw_type=tw_type, regressors=subset, 
                                                                                      exp_nr=exp_nr, tw_per_el=tw_per_el, 
                                                                                      use_all_els=use_all_els, k=5, 
                                                                                      elec_nrs=get_best_electrodes_2(
                                                                                          subset, tw_type, p_elec=percent), 
                                                                                      verbose=False)
                exp_nr += 1
                
                
    return results_ao, results_gb

**Current trial regressors:**

In [65]:
# results_ao_2, results_gb_2 = all_combinations_best_elecs_2(all_datasets)

1.5s after outcome reveal
['loss_ind', 'regret', 'rpe', 'win_ind']
['loss_ind', 'regret']
['loss_ind', 'rpe']
['loss_ind', 'win_ind']
['regret', 'rpe']
['regret', 'win_ind']
['rpe', 'win_ind']
['loss_ind', 'regret', 'rpe']
['loss_ind', 'regret', 'win_ind']
['loss_ind', 'rpe', 'win_ind']


C:\Users\MariLiis\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
C:\Users\MariLiis\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
C:\Users\MariLiis\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
C:\Users\MariLiis\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
C:\Users\MariLiis\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
C:\Users\MariLiis\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)


['regret', 'rpe', 'win_ind']
between game presentation and buttonpress
['exputil', 'gamble_ind', 'risk', 'winprob']
['exputil', 'gamble_ind']
['exputil', 'risk']
['exputil', 'winprob']
['gamble_ind', 'risk']
['gamble_ind', 'winprob']
['risk', 'winprob']
['exputil', 'gamble_ind', 'risk']
['exputil', 'gamble_ind', 'winprob']
['exputil', 'risk', 'winprob']


C:\Users\MariLiis\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
C:\Users\MariLiis\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
C:\Users\MariLiis\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
C:\Users\MariLiis\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
C:\Users\MariLiis\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
C:\Users\MariLiis\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)


['gamble_ind', 'risk', 'winprob']


In [66]:
# save_var((default_to_regular(results_ao_2), default_to_regular(results_gb_2)), "all_combinations_best_elecs_2")

In [70]:
results_gb_2[('gamble_ind', 'risk', 'winprob')].keys()

dict_keys([(1, 'all', 0.2), (1, 'best', 0.2), (1, 'all', 0.4), (1, 'best', 0.4), (1, 'all', 0.6), (1, 'best', 0.6), (1, 'all', 0.8), (1, 'best', 0.8), (1, 'all', 1.0), (1, 'best', 1.0), (2, 'all', 0.2), (2, 'best', 0.2), (2, 'all', 0.4), (2, 'best', 0.4), (2, 'all', 0.6), (2, 'best', 0.6), (2, 'all', 0.8), (2, 'best', 0.8), (2, 'all', 1.0), (2, 'best', 1.0), (3, 'all', 0.2), (3, 'best', 0.2), (3, 'all', 0.4), (3, 'best', 0.4), (3, 'all', 0.6), (3, 'best', 0.6), (3, 'all', 0.8), (3, 'best', 0.8), (3, 'all', 1.0), (3, 'best', 1.0)])

In [71]:
results_gb_2[('gamble_ind', 'risk', 'winprob')][(1, 'all', 0.2)].keys()

dict_keys(['Y_features', 'nr', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10'])

**Previous trial regressors:**

In [110]:
# results_ao_prev_2, results_gb_prev_2 = all_combinations_best_elecs_2(all_datasets, outcome_prev_subsets, gamebtn_prev_subsets)

1.5s after outcome reveal
['previous_loss_ind', 'previous_regret', 'previous_rpe', 'previous_win_ind']
['previous_loss_ind', 'previous_regret']
['previous_loss_ind', 'previous_rpe']
['previous_loss_ind', 'previous_win_ind']
['previous_regret', 'previous_rpe']
['previous_regret', 'previous_win_ind']
['previous_rpe', 'previous_win_ind']
['previous_loss_ind', 'previous_regret', 'previous_rpe']
['previous_loss_ind', 'previous_regret', 'previous_win_ind']
['previous_loss_ind', 'previous_rpe', 'previous_win_ind']
['previous_regret', 'previous_rpe', 'previous_win_ind']
between game presentation and buttonpress
['previous_exputil', 'previous_gamble_ind', 'previous_risk', 'previous_winprob']
['previous_exputil', 'previous_gamble_ind']
['previous_exputil', 'previous_risk']
['previous_exputil', 'previous_winprob']
['previous_gamble_ind', 'previous_risk']
['previous_gamble_ind', 'previous_winprob']
['previous_risk', 'previous_winprob']
['previous_exputil', 'previous_gamble_ind', 'previous_risk']
[

In [111]:
# save_var((default_to_regular(results_ao_prev_2), default_to_regular(results_gb_prev_2)), "all_combinations_best_elecs_prev_2")